PyPortfolioOpt is a library that implements portfolio optimization methods, including
classical mean-variance optimization techniques and Black-Litterman allocation, as well as more
recent developments in the field like shrinkage and Hierarchical Risk Parity.

It is **extensive** yet easily **extensible**, and can be useful for either a casual investors, or a professional looking for an easy prototyping tool. Whether you are a fundamentals-oriented investor who has identified a
handful of undervalued picks, or an algorithmic trader who has a basket of
strategies, PyPortfolioOpt can help you combine your alpha sources
in a risk-efficient way.

**PyPortfolioOpt has been [published](https://joss.theoj.org/papers/10.21105/joss.03066) in the Journal of Open Source Software 🎉**

PyPortfolioOpt is now being maintained by [Tuan Tran](https://github.com/88d52bdba0366127fffca9dfa93895).

Head over to the **[documentation on ReadTheDocs](https://pyportfolioopt.readthedocs.io/en/latest/)** to get an in-depth look at the project, or check out the [cookbook](https://github.com/robertmartin8/PyPortfolioOpt/tree/master/cookbook) to see some examples showing the full process from downloading data to building a portfolio.

<center>
<img src="https://github.com/robertmartin8/PyPortfolioOpt/blob/master/media/conceptual_flowchart_v2.png?raw=true" style="width:70%;"/>
</center>

In [1]:
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Read in price data
df = pd.read_csv("stock_prices.csv", parse_dates=True, index_col="date")

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 30.5%
Annual volatility: 22.2%
Sharpe Ratio: 1.28


(0.30477686728199166, 0.2216556692240296, 1.2847714127003214)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize
from pypfopt import EfficientFrontier, risk_models, expected_returns
import cvxpy as cp
import random

tickers = ['AAPL', 'MSFT', 'GOOGL', 'GLMD', 'AMZN', 'NFLX', 'TSLA', 'NVDA', 'BABA', 'JPM']
data = yf.download(tickers, start='2020-01-01', end='2023-01-01')['Adj Close']

# 2. Calculate expected returns and covariance matrix
returns = data.pct_change().dropna()

# Update tickers to match the remaining assets after dropna()
tickers = returns.columns
mean_returns = returns.mean()
cov_matrix = returns.cov()

# Symmetrize the covariance matrix to ensure it's valid
cov_matrix = (cov_matrix + cov_matrix.T) / 2

num_assets = len(tickers)



# Constraint: sum of weights = 1
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

# Bounds: No short selling (weights >= 0)x
bounds = tuple((0, 1) for asset in range(num_assets))

# Optimization



# 5. Minimum Variance Portfolio (Efficient Frontier)
S = risk_models.sample_cov(returns)
mu = expected_returns.mean_historical_return(data)
ef = EfficientFrontier(mu, cov_matrix)

weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

[*********************100%***********************]  10 of 10 completed

Expected annual return: 56.1%
Annual volatility: 4.0%
Sharpe Ratio: 13.54


(0.5606419210480541, 0.03991713456904051, 13.544106481715566)